In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
game_detail = "../dataset/game_detail_v2.json"
user_friend = "../dataset/user_friend_list.json"
user_idx = "../dataset/user_idx.json"
user_owned_games = "../dataset/user_owned_games.json"
user_recently_played_games = "../dataset/user_recently_played_games.json"
user_summary = "../dataset/user_summary.json"

# 1. Convert json to dataframe

In [3]:
game_detail_df = pd.read_json(game_detail, lines = True)
user_frined_df = pd.read_json(user_friend, lines = True)
user_idx_df = pd.read_json(user_idx, lines = True)
user_owned_games_df = pd.read_json(user_owned_games, lines = True)
user_recent_df = pd.read_json(user_recently_played_games, lines = True)
user_summary_df  = pd.read_json(user_summary, lines = True)

In [4]:
print("user_idx columns         :", user_idx_df.columns.values)
print("user_owned_games columns :", user_owned_games_df.columns.values)
print("user_recent_df columns   :", user_recent_df.columns.values)
print("user_summary columns     :", user_summary_df.columns.values)
print("game_detail_df columns   :", game_detail_df.columns.values)

user_idx columns         : ['user_idx' 'user_id']
user_owned_games columns : ['steamid' 'game_count' 'games']
user_recent_df columns   : ['steamid' 'total_count' 'games']
user_summary columns     : ['steamid' 'primaryclanid' 'realname' 'personaname' 'personastate'
 'personastateflags' 'communityvisibilitystate' 'avatarhash'
 'profilestate' 'profileurl' 'loccityid' 'timecreated' 'avatar'
 'locstatecode' 'commentpermission' 'avatarfull' 'avatarmedium'
 'loccountrycode' 'gameextrainfo' 'gameid' 'gameserverip'
 'gameserversteamid' 'lobbysteamid' 'lastlogoff']
game_detail_df columns   : ['steam_appid' 'achievements' 'price_overview' 'demos'
 'detailed_description' 'screenshots' 'genres' 'required_age'
 'about_the_game' 'short_description' 'developers' 'type'
 'supported_languages' 'website' 'publishers' 'pc_requirements'
 'recommendations' 'is_free' 'legal_notice' 'background'
 'controller_support' 'packages' 'categories' 'support_info'
 'content_descriptors' 'name' 'platforms' 'package_gro

# 2. Preprocessing

## (1) convert user id to index

In [76]:
user_game_df = user_idx_df

In [77]:
user_owned_games_df.head(100)

,steamid,game_count,games
0,76561197960434624,1047,"[{'playtime_linux_forever': 0, 'playtime_forev..."
1,76561198015158496,[0],[]
2,76561197972495328,[0],[]
3,76561198028573552,[0],[]
4,76561197970323424,[0],[]
...,...,...,...
13037,76561198950450384,[0],[]
13038,76561198974830112,[0],[]
13039,76561198985708688,[0],[]
13040,76561198993673408,[0],[]


## (2) make data frame of user and playtime of each game

In [78]:
import time
start = time.time()
for idx, line in enumerate(open(user_owned_games, 'r')):
    if idx % 100 == 0:
        print("time elapsed : ", time.time() - start)
    temp = json.loads(line)
    temp_data = pd.json_normalize(temp["games"], max_level = 1)
    temp_idx = user_idx_df[user_idx_df.user_id==user_owned_games_df.steamid[idx]].index.values
#     print(user_owned_games_df.steamid[idx])
#     if (temp_data[""])
#     print(user_owned_games_df.steamid[idx])
    if(temp_idx.size == 0 or temp_data.size == 0):
        continue
    if(user_owned_games_df.game_count[idx] > 1000):
        continue ;
    for app_id, play_forever in zip(temp_data["appid"], temp_data["playtime_forever"]):
        if play_forever==0:
            continue
        if not app_id in user_game_df.columns:
            user_game_df[app_id] = 0
        user_game_df.loc[temp_idx, app_id] = play_forever
#    user_game_df.
#     user_game_df.insert(temp_idx[0],
#                     temp_data.appid,
#                     temp_data.playtime_forever)

time elapsed :  0.005989789962768555
time elapsed :  292.9777331352234
time elapsed :  967.260692358017
time elapsed :  1482.5300981998444
time elapsed :  1791.589519739151
time elapsed :  2037.4277067184448
time elapsed :  2166.429635286331
time elapsed :  2367.383463859558
time elapsed :  2494.3921513557434
time elapsed :  2626.9598417282104
time elapsed :  2773.313987970352
time elapsed :  2915.062644481659
time elapsed :  3048.533887386322
time elapsed :  3154.699777364731
time elapsed :  3278.240430355072
time elapsed :  3422.0797069072723
time elapsed :  3595.378268957138
time elapsed :  3690.423092842102
time elapsed :  3757.9937307834625
time elapsed :  3799.1244575977325
time elapsed :  3900.032322883606
time elapsed :  4004.0810050964355
time elapsed :  4125.309991121292
time elapsed :  4204.530155181885
time elapsed :  4347.822320461273
time elapsed :  4453.610534191132
time elapsed :  4576.439901351929
time elapsed :  4619.103406906128
time elapsed :  4673.624705553055
time

In [81]:
user_game_df.head(25)

,user_idx,user_id,10,20,30,40,50,60,70,130,...,809320,575990,333690,803150,338630,436280,638850,581380,645450,774711
0,0,76561197960434624,32,3,6,0,52,58,412,1,...,0,0,0,0,0,0,0,0,0,0
1,1,76561198015158496,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,76561197972495328,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,76561198028573552,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,76561197970323424,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,76561198024402256,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,76561197968459472,40,106,0,0,0,9,0,35,...,0,0,0,0,0,0,0,0,0,0
7,7,76561197970565168,0,0,0,0,0,0,35,0,...,0,0,0,0,0,0,0,0,0,0
8,8,76561197980258576,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,76561198000175600,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
user_game_df.to_csv("user_game.csv", mode='w')

## (3) Use collaborate filletering to make recommender  system